# PySMO Tutorial

**Python-based Surrogate Modelling Objects** (PySMO) provides tools for generating different types of reduced order models. PySMO currently provides tools for sampling and surrogate model generation.

## Installation

**PySMO** is installed by default as part of IDAES. For instructions on installing IDAES, see the [online documentation](https://idaes-pse.readthedocs.io/en/stable/).


## One-Shot Sampling with PySMO

The PySMO package offers five common sampling methods for one-shot design:

* Latin Hypercube Sampling (LHS)
* Full-Factorial Sampling
* Halton Sampling
* Hammersley Sampling
* Centroidal voronoi tessellation (CVT) sampling

PySMO provides two modes for data sampling: creation and selection.
- In creation mode, PySMO creates a specified number of sample points from the bounds provided by the user.
- In selection mode, PySMO selects a specified number of data points from a user-supplied dataset or file.

### Generating samples: 
For demonstration purposes, let us consider a problem for which we need twenty-five (25) samples of temperature and pressure from within the ranges T = 273K - 373K, P = 1 MPa - 50 MPa. Let us generate these samples in PySMO.

#### Step 1: Import PySMO's sampling tool
For this demonstration, we will attempt to generate the samples using the Hammersley sampling method.

In [1]:
from idaes.surrogate.pysmo.sampling import HammersleySampling

#### Step 2: Specify sampling information and initialize class

All the sampling tools (except full-factorial sampling) require the same keyword arguments:

          -  data_input             : must be a list of lists containing the problem bounds (when creating points), 
                                      or an input dataset (when selecting points from a dataset)
          -  number_of_samples      : number of samples to be created or selected.
          -  sampling_type          : "creation" or "selection".

For full factorial sampling, the user needs to enter a list of points in each dimension in place of the number of samples. Full-factorial sampling requires other inputs - details may be found in the [documentation](https://idaes-pse.readthedocs.io/en/stable/surrogate/pysmo/index.html).

For our example, we will create the bounds and then initialize the class with the number of samples.

In [2]:
bounds_info = [[273, 1],[373, 50]]
init_data = HammersleySampling(data_input=bounds_info, number_of_samples=25, sampling_type="creation")

#### Step 3: Create the samples
The samples are created by calling the ``sample_points`` method on the initialized class.

In [3]:
samples = init_data.sample_points()
print(samples)

Simple as that, the samples have been created!

Now, let us visualize the samples in a 2-D plot.

#### Step 4: Visualize samples with matplotlib

In [4]:
from matplotlib import pyplot as plt
plt.plot(samples[:, 0], samples[:, 1], 'o')
plt.xlabel(r'Temperature', fontsize=11)
plt.xlabel(r'Pressure', fontsize=11)
plt.xlim(272, 374)
plt.ylim(0, 50)
plt.show()

## Generating surrogates with PySMO

PySMO currently provides tools for generating three types of surrogates:

- Polynomial surrogates
- Radial basis function (RBF) surrogates, and
- Kriging surrogates

Details about thee various methods may be found in the [documentation](https://idaes-pse.readthedocs.io/en/stable/surrogate/pysmo/index.html).

### Generating polynomial models

The ``PolynomialRegression`` class trains polynomial models from data.

As an example, let us generate a surrogate for the Brainin function. 

The true Brainin function is given by the expression:
 
 \begin{gather}
\hat{y}(x_{1},x_{2})=\left(x_{2}-\frac{5.1x_{1}^{2}}{4\pi^{2}}+\frac{5x_{1}}{\pi}-6\right)^{2}+10\left[\left(1-\frac{1}{8\pi}\right)\cos\left(x_{1}\right)+1\right]+5x_{1}\nonumber \\
x_{1}\in\left[-5,10\right];x_{2}\in\left[0,15\right]
\end{gather}

We have generated 30 points from the function and saved the information in a text file called "brainin_30.txt". We will use this data to train a simple polynomial model. The data is in XY format, with the outputs $y$ in the third column.

#### Step 1: Import and visualize the data

In [5]:
import numpy as np
brainin_data = np.loadtxt('brainin_30.txt')
print(brainin_data,'\n\nDataset shape:', brainin_data.shape)

Let us visualize the data:

In [10]:
from mpl_toolkits.mplot3d import Axes3D 
from matplotlib import pyplot as plt
fig1 = plt.figure(figsize=(6, 4), tight_layout=True)
ax = fig1.add_subplot(111, projection='3d')
ax.scatter3D(brainin_data[:, 0], brainin_data[:, 1], brainin_data[:, 2], cmap=brainin_data[:, 2])
ax.set_xlabel(r'$x_{1}$', fontsize=11)
ax.set_ylabel(r'$x_{2}$', fontsize=11)
ax.set_zlabel(r'$y$', fontsize=11)

#### Step 2: Import the polynomial model tool

In [2]:
from idaes.surrogate.pysmo.polynomial_regression import PolynomialRegression

#### Step 3: Specify the regression settings and initialize the PolynomialRegression class
    
The PolynomialRegression class takes a keyword arguments:

      -  original_data_input           : The dataset for regression training. training_data is expected to contain xy_data, 
                                         with the output values (y) in the last column.
      -  regression_data_input         : same as above
      -  maximum_polynomial_order      : maximum order of the polynomial to be generated  

It also takes a number of optional arguments:

      - multinomials                  : True/False option for specifying second-order bi-variate terms. default is False
      - training_split                : The training/cross-validation split of training data. Must be between 0 and 1. 
                                        Default is 0.75
      - fname                         : Filename for saving results (.pickle extension). 
      - overwrite                     : Option determining whether any existing file with the same name supplied in 'fname'  
                                        should be overwritten.

For this example, let us consider a 4th order polynomial with interaction terms. We will split the data 80/20 betweeen training and cross-validation.

In [15]:
poly_class = PolynomialRegression(original_data_input=brainin_data, regression_data_input=brainin_data, 
                                  maximum_polynomial_order=4, multinomials=1, training_split=0.8, number_of_crossvalidations=10, overwrite=True)

#### Step 4: Extract variable names
Next, we extract Pyomo variable names from the dataset. This should be done always.

In [16]:
vars = poly_class.get_feature_vector()

We can view the variables using Pyomo's pprint function:

In [17]:
vars.pprint()

#### Step 5: Specify additional regression terms, if required.

This is one of the unique features of PySMO -  it allows the user to specify additional regression features if they want.
The additional features must be specified in terms of the Pyomo variables created when calling the ``get_feature_vector()`` 

For this example, let us create three additional features: $x_{1}^{2}x_{2}^{2}$, $exp(x_1)$ and $exp(x_2)$. We do this by calling the ``set_additional_terms`` function:

In [18]:
from pyomo.environ import exp
poly_class.set_additional_terms([vars[0] * vars[0] * vars[1] * vars[1], exp(vars[0]), exp(vars[1])])

That's it - those features will now exist in the model. 

Note that ``set_additional_terms`` an optional call - the regression process works just fine without it.

#### Step 6: Train the surrogate and view results
Next, we train the polynomial surrogate by calling ``poly_training``:

In [19]:
result = poly_class.poly_training()

The polynomial model seems to fit well based on the $R^2$. It should be noted that the metrics are only an indication of how well of how well the model fit the training data - the user needs to verify the model's performance on a test data set if possible.

We can view the parity and residual plots for the fit:

In [20]:
poly_class.parity_residual_plots(result)

PySMO is also able to compute the confidence intervals on the regression coefficients obtained by calling ``confint_regression()``. This is left as an exercise for the user.

#### Step 8 (Optional): Generate Pyomo expression 

If the user wishes, they can generate the Pyomo expression for the polynomial fit using PySMO's ``generate_expression``.

To do this, the user must pass in a list of Pyomo variables corresponding to each variable in the input dataset. The user should note that ``generate_expression`` should be called on the result object, not the class object.

As a demonstration, let us create the variables $x_1$ and $x_2$ and generate the pyomo expression based on them:

In [21]:
from pyomo.environ import Var, ConcreteModel
m = ConcreteModel()
m.x = Var([1, 2])
print(result.generate_expression([m.x[1], m.x[2]]))

#### Step 9 (Optional): Predict output at any unsampled point

Based on the model we trained, we can predict the surrogate value at any previously unsampled point. 

Let us evaluate the surrogate at three points:

- $x_{1}=5$, $x_{2}=8$ (true function value: 57.9908)
- $x_{1}=-3$, $x_{2}=10$ (true function value: 4.2461)
- $x_{1}=-2$, $x_{2}=3$. (true function value: 50.8899)

We will pass the points in as an array.

In [22]:
unsampled_points = np.array([ [5, 8], [-3, 10], [-2, 3]])
ys = poly_class.poly_predict_output(result, unsampled_points)
print(ys)

The model performs fairly well in predicting the value at two of our sampled points but is off on the value at [-3, 2]. For better model performance, additional training data is needed in this region. We will leave this to the user to try.

Further information about using PySMO's polynomial regression tool can be found in the [documentation](https://idaes-pse.readthedocs.io/en/stable/surrogate/pysmo/pysmo_polyregression.html).

### Generating RBF models

The ``RadialBasisFunction`` class trains RBF models from data. For details about RBF models, the user should consult the documentation.

As an example, we will again consider the Brainin function. The same dataset loaded previously will be used.

#### Step 1: Import the data and the RBF tool

In [23]:
import numpy as np
brainin_data = np.loadtxt('brainin_30.txt')
from idaes.surrogate.pysmo.radial_basis_function import RadialBasisFunctions

#### Step 2: Specify the RBF settings and initialize the RadialBasisFunctions class
    
The RadialBasisFunctions class takes a number of keyword arguments:

      -  XY_data                       : The dataset forRBF training. training_data is expected to contain xy_data, 
                                         with the output values (y) in the last column.


It also takes a number of optional arguments:

      -  regularization                : Boolean variable determining whether regularization is done. Default is True.
      -  basis_function                : Basis function transformation to be applied to the training data. PySMO offers 
                                         six basis function types including the Gaussian and Spline transformations. User 
                                         should consult documentation for full list of options.   
      -  fname                         : Filename for saving (.pickle extension)
      - overwrite                      : Option determining whether any existing file with the same name supplied in 'fname'  
                                         should be overwritten.

For this demonstration, we will train an RBF model with a Gaussian basis function:

In [24]:
rbf_class = RadialBasisFunctions(XY_data=brainin_data, basis_function='gaussian', overwrite=True)

#### Step 3: Extract variable names
Next, we extract Pyomo variable names from the dataset.

In [25]:
vars = rbf_class.get_feature_vector()

#### Step 4: Train the RBF surrogate
Next, we train the RBF surrogate by calling ``rbf_training``:

In [26]:
result = rbf_class.rbf_training()

#### Step 5: View model metrics
    
We can view the Root Mean Square Error (RMSE) and $R^2$ values of the RBF fit based on the training data:

In [27]:
print('R2: ', result.R2, '\nRMSE: ', result.rmse)

#### Step 6 (Optional): Generate Pyomo expression output at any unsampled point


Based on the model we trained, we can predict the surrogate value at any previously unsampled point. We do this by calling the function ``rbf_predict_output``.

Let us again evaluate the RBF surrogate at the same set of points we considered for the polynomial model:

- $x_{1}=5$, $x_{2}=8$   (true function value: 57.9908)
- $x_{1}=-3$, $x_{2}=10$ (true function value: 4.2461)
- $x_{1}=-2$, $x_{2}=3$  (true function value: 50.8899)

In [28]:
unsampled_points = np.array([ [5, 8], [-3, 10], [-2, 3]])
ys = rbf_class.rbf_predict_output(result, unsampled_points)
print(ys)

The results from the RBF surrogate are slightly better than those obtained form the polynomial.


For RBF models, the Pyomo expression is generated by calling ``rbf_generate_expression`` on the results object, while parity plots may be viewed with the ``parity_residual_plots`` method.

Further information about using PySMO's RBF tool and features can be found in the [documentation](https://idaes-pse.readthedocs.io/en/stable/surrogate/pysmo/pysmo_radialbasisfunctions.html).

### Training Kriging models

The ``KrigingModel`` class trains Kriging from data. For details about Kriging models, users should consult the documentation.

As an example, we will again consider the Brainin function. The same dataset loaded previously will be used.

#### Step 1: Load the data and import the Kriging tool

In [29]:
import numpy as np
brainin_data = np.loadtxt('brainin_30.txt')
from idaes.surrogate.pysmo.kriging import KrigingModel

#### Step 2: Specify the Kriging settings and initialize the KrigingModel class
    
The KrigingModel class takes a number of keyword arguments:

      -  XY_data                       : The dataset for Kriging training. training_data is expected to contain xy_data, 
                                         with the output values (y) in the last column.


It also takes a number of optional arguments:

      -  regularization                : Boolean variable determining whether regularization is done. Default is True.
      -  numerical_gradients           : Boolean variable which determines whether numerical gradients are used when
                                         solving the max. likelihood optimization problem. Default is True.
      -  fname                         : Filename for saving (.pickle extension)
      - overwrite                      : Option determining whether any existing file with the same name supplied in 'fname'  
                                         should be overwritten.


For this demonstration, we will train a Kriging model with regularization:

In [30]:
krg_class = KrigingModel(XY_data=brainin_data, overwrite=True)

#### Step 3: Extract variable names (optional)
Next, we extract Pyomo variable names from the dataset.

In [31]:
vars = krg_class.get_feature_vector()

#### Step 4: Train the Kriging surrogate
Next, we train the RBF surrogate by calling ``kriging_training``:

In [32]:
result = krg_class.kriging_training()

The returned information correspond to the Kriging model parameters.

As we can see, the optimization problem was solved using SciPy's L-BFGS algorithm which makes use of gradients. A different algorithm  (Basinhopping) is used when no numerical gradients are computed (when numerical_gradients is set to False). The user should try this.

#### Step 5: View model metrics
    
We can view the RMSE and $R^2$ values of the Kriging fit based on the training data:

In [33]:
print('R2: ', result.training_R2, '\nRMSE: ', result.training_rmse)

#### Step 6 (Optional): Generate Pyomo expression output at any unsampled point


Again, based on the model we trained, we evaluate the surrogate at a set of  off-design points:

- $x_{1}=5$, $x_{2}=8$   (true function value: 57.9908)
- $x_{1}=-3$, $x_{2}=10$ (true function value: 4.2461)
- $x_{1}=-2$, $x_{2}=3$  (true function value: 50.8899)

We do this by calling the function ``kriging_predict_output``:

In [34]:
unsampled_points = np.array([ [5, 8], [-3, 10], [-2, 3]])
ys = krg_class.kriging_predict_output(result, unsampled_points)
print(ys)

The Kriging model performs very well, predicting all three points fairly accurately. 

For RBF models, the Pyomo expression is generated by calling ``kriging_generate_expression`` on the results object: 

In [35]:
print(result.kriging_generate_expression([m.x[1], m.x[2]]))

As we can see, expressing a Kriging model algebraically is pretty complicated!

Parity plots for the Kriging model may be viewed with the ``parity_residual_plots`` method.

Further information about using PySMO's Krigng tool and features can be found in the [documentation](https://idaes-pse.readthedocs.io/en/stable/surrogate/pysmo/pysmo_kriging.html).

## Summary

PySMO allows IDAES users to sample design spaces and generate different types of surrogate models. Further information about PySMO's capabilities may be found in the documentation.